# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

### Topic: Building Recommendation Systems for eCommerce
### Project link:

https://drive.google.com/file/d/1O5wSSYrw84Dpv2iaR5XzMz1ll0LgXBYK/view

### Who

The audience is the data science and product teams at a mid/large e-commerce or gaming platform 
companies like Amazon, Target, Olist, or Nintendo, Steam, or XBox. Specifically, I will target the 
lead data scientist who oversees recommendation algorithms, and a product manager responsible for personalization. 
These people are directly responsible for improving user satifaction with their platforms and conversion through
better recommendations, so they would benefit most from exploratory insights that could uncover overlooked patterns 
or opportunities for better user data modelling.

### What

This project investigates how machine learning can enhance product recommendation systems in ecommerce by 
analysing real world data on user reivews, puchase behavior, product attributes, and sales. Using the following three
datasets: Steam video game recommendations, Worldwide video game sales, and Olist retail data, I will explore the product categories
and game genres that are most popular across different regions. I will also look at which features (ratings, timing, genre, price)
correlate strongly with success measures like purchases or high review ratings. The goal is to uncover meaningful patterns that can inform
how we model user preferences and improve personalization within online sales.

### How

I can deliver a structured report (slideument) or Jupyter notebook with clear annotated visuals, key takeaways, and a 
short conclusive summary of insights from the data. I'll compare how user interaction data, sales performance, and transactional behavior 
vary across the gaming and retail domains to answer the question: What product categories or game genres are most popular across regions? The intent is not to pitch a model yet, but to highlight promising trends—that could guide future
recommendation system development and prioritize which features to engineer or model next.

# 3. Homework - work with your own data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import kagglehub
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

In [ ]:
# helper function to display mean, median, mode of a df
def df_mmm_stats(df):
    numeric_df = df.select_dtypes(include="number")

    means = numeric_df.mean()
    medians = numeric_df.median()
    modes = numeric_df.mode().iloc[0] # get da first mode, don"t really care baout the others

    summary_stats = pd.DataFrame({"mean": means, "median": medians,"mode": modes}).round(2) # two decimal places

    return summary_stats


In [ ]:
olist_dataset_link = "olistbr/brazilian-ecommerce"
vgrecs_dataset_link = "antonkozyriev/game-recommendations-on-steam"
vgsales_dataset_link = "ashaheedq/video-games-sales-2019"

olist_path = kagglehub.dataset_download(olist_dataset_link)
vgrec_path = kagglehub.dataset_download(vgrecs_dataset_link)
vgsales_path = kagglehub.dataset_download(vgsales_dataset_link)

print(olist_path)
print(vgrec_path)
print(vgsales_path)


## Loading and Merging Datasets

In [ ]:
# Video Game Sales DFs
vgsales_df_raw = pd.read_csv(os.path.join(vgsales_path, "vgsales-12-4-2019-short.csv"))

# Video Game Recommendation DFs
vgrecs_recs_df_raw = pd.read_csv(os.path.join(vgrec_path, "recommendations.csv")) # merge with users based on "user_id" into vgrecs_merged_df_raw, add "users_" prefix to users.csv columns
vgrecs_users_df_raw = pd.read_csv(os.path.join(vgrec_path, "users.csv"))
vgrecs_games_df_raw = pd.read_csv(os.path.join(vgrec_path, "games.csv")) # merge with vgrecs_merged_df_raw on "app_id"

vgrecs_merged_df = vgrecs_recs_df_raw.merge(vgrecs_users_df_raw, on="user_id", how="left")
vgrecs_merged_df = vgrecs_merged_df.merge(vgrecs_games_df_raw, on="app_id", how="left")

# Olist DFs
olist_products_df_raw = pd.read_csv(os.path.join(olist_path, "olist_products_dataset.csv"))
olist_translation_df_raw = pd.read_csv(os.path.join(olist_path, "product_category_name_translation.csv"))
olist_reviews_df_raw = pd.read_csv(os.path.join(olist_path, "olist_order_reviews_dataset.csv"))
olist_orders_df_raw = pd.read_csv(os.path.join(olist_path, "olist_orders_dataset.csv"))
olist_geolocation_df_raw = pd.read_csv(os.path.join(olist_path, "olist_geolocation_dataset.csv"))
olist_customers_df_raw = pd.read_csv(os.path.join(olist_path, "olist_customers_dataset.csv"))
olist_items_df_raw = pd.read_csv(os.path.join(olist_path, "olist_order_items_dataset.csv"))
olist_payments_df_raw = pd.read_csv(os.path.join(olist_path, "olist_order_payments_dataset.csv"))
olist_sellers_df_raw = pd.read_csv(os.path.join(olist_path, "olist_sellers_dataset.csv"))

# Merge the product categories with their english translations
olist_products_df = (olist_products_df_raw.merge(olist_translation_df_raw, how="left", on="product_category_name"))
# Merge the products with the order items so we know what the orders contained
olist_order_lines_df = (olist_items_df_raw.merge(olist_products_df, how="left", on="product_id"))
# Merge the orders with the orders" customer ID, date, and order ID so we can merge it with the customer locations later
olist_order_lines_df = olist_order_lines_df.merge(olist_orders_df_raw[["order_id", "customer_id", "order_purchase_timestamp"]], how="left", on="order_id")
# Merge the orders with the customer data
olist_order_lines_df = olist_order_lines_df.merge(olist_customers_df_raw[["customer_id", "customer_unique_id", "customer_state"]],how="left",on="customer_id")
olist_merged_df = olist_order_lines_df.merge(olist_reviews_df_raw[["order_id", "review_score", "review_creation_date", "review_answer_timestamp"]], how="left", on="order_id")


# Processing the Video Game Sales Dataset

## Features (from kaggle)
| Column Name    | Description                                             |
|----------------|---------------------------------------------------------|
| Rank           | Ranking of overall sales                                |
| Name           | Name of the game                                        |
| Platform       | Platform of the game (i.e. PC, PS4, XOne, etc.)         |
| Genre          | Genre of the game                                       |
| ESRB Rating    | ESRB Rating of the game                                 |
| Publisher      | Publisher of the game                                   |
| Developer      | Developer of the game                                   |
| Critic Score   | Critic score of the game out of 10                      |
| User Score     | Users score the game out of 10                          |
| Total Shipped  | Total shipped copies of the game                        |
| Global_Sales   | Total worldwide sales (in millions)                     |
| NA_Sales       | Sales in North America (in millions)                    |
| PAL_Sales      | Sales in Europe (in millions)                           |
| JP_Sales       | Sales in Japan (in millions)                            |
| Other_Sales    | Sales in the rest of the world (in millions)            |
| Year           | Year of release of the game                             |


In [ ]:
# Summary Stats
vgsales_df_raw.info()
vgsales_df_raw.describe(include="all")


In [ ]:
# view the df
vgsales_df_raw.sample(10)

In [ ]:
# Checking for duplicate rows
print("Duplicate rows in vgsales:", vgsales_df_raw.duplicated().sum())

# No duplicates yeehaw

In [ ]:
# Mean, median, mode
df_mmm_stats(vgsales_df_raw)

In [ ]:
# Identifying nulls
print("Video Game Sale Null Counts:")
print(vgsales_df_raw.isna().sum())
print(vgsales_df_raw.shape)
sales_scores_cols = [
    "Critic_Score", "User_Score",
    "Total_Shipped", "Global_Sales",
    "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"
]

# Count rows where all of the performance column values (sales_scores_cols) are null
all_null_sales_scores = vgsales_df_raw[sales_scores_cols].isna().all(axis=1).sum()
print("Rows where all sales/scores are MISSING:", all_null_sales_scores)


# Of the 55792 rows in the Video Game Sale dataset, 32794 rows have missing performance metrics for
# the whole row (e.g. scores, sales).This reduces the number of usable rows in the dataset (with at 
# least one performance metric) down to 55792 - 32794 = 22998 rows.
# RANK CAN BE USED AS PERFORMANCE METRIC I GUESS


# Count rows where all of the performance column values are present
all_present_sales_scores = vgsales_df_raw[sales_scores_cols].notna().all(axis=1).sum()
print("Rows with all sales/scores PRESENT:", all_present_sales_scores)


# Count rows where both Critic_Score and User_Score are NOT null
both_scores_present = vgsales_df_raw[vgsales_df_raw["Critic_Score"].notna() & vgsales_df_raw["User_Score"].notna()].shape[0]
print("Rows with BOTH Critic_Score and User_Score present:", both_scores_present)


# Drop rows with null "Year", should only be 17 rows discarded
vgsales_df_cleaned = vgsales_df_raw.dropna(subset=["Year"])

# Fill null ESRB ratings with "RP" --> rating pending
vgsales_df_cleaned["ESRB_Rating"] = vgsales_df_cleaned["ESRB_Rating"].fillna("RP")

# Null Developer values to "Unknown"
vgsales_df_cleaned["Developer"] = vgsales_df_cleaned["Developer"].fillna("Unknown")

# Null sales values become 0:
sales_cols = ["Global_Sales", "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"]
vgsales_df_cleaned[sales_cols] = vgsales_df_cleaned[sales_cols].fillna(0)

In [ ]:
# Handling inconsistencies

# Drop rows for any years that are in the future (>2025) or past
vgsales_df_cleaned = vgsales_df_cleaned[vgsales_df_cleaned["Year"] <= 2025]


# Ensure that the Global_Sales is not less than the sum of NA_Sales, PAL_Sales, JP_Sales, and Other_Sales
regional_cols = ["NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"]
# sum the sales of the regions
regional_sum = (vgsales_df_cleaned["NA_Sales"] + vgsales_df_cleaned["PAL_Sales"] + vgsales_df_cleaned["JP_Sales"] + vgsales_df_cleaned["Other_Sales"])
# drop row if the global sales value is less thaen the regional sales value
num_dropped_sales_rows = (vgsales_df_cleaned["Global_Sales"] < regional_sum).sum()
print(f"Dropped {num_dropped_sales_rows} rows where the global sales value was less than the sum of sales in America, Europe, Japan, and other.")
vgsales_df_cleaned = vgsales_df_cleaned[vgsales_df_cleaned["Global_Sales"] >= regional_sum]

# Drop "total shipped" column, it has too many nulls and doesn't tell us anything helpful
vgsales_df_cleaned = vgsales_df_cleaned.drop("Total_Shipped", axis=1)

In [ ]:
# Check nulls again:
print("Video Game Sale Null Counts:")
print(vgsales_df_cleaned.isna().sum())
vgsales_df_cleaned.sample(10)

In [ ]:
# View the unique values to make sure they make sense
categorical_cols = [
    "Platform",
    "Genre",
    "ESRB_Rating",
    "Publisher",
    "Developer"
]
for col in categorical_cols:
    unique_vals = vgsales_df_cleaned[col].unique()
    print(f"\nColumn: {col}")
    print(f"Number of unique values: {len(unique_vals)}")
    print(unique_vals)

In [ ]:
# Check how many times each type of platform so we can figue out how to encode it
print(vgsales_df_cleaned["Platform"].value_counts().to_string())

# Probably top n encoding, up to 15 of the top platforms


In [ ]:
# Create a new df with encoded categorical variables for modelling later
vgsales_df_cleaned_encoded = vgsales_df_cleaned.copy()

# One hot encode Genre and ESRB_Rating
vgsales_df_cleaned_encoded = pd.get_dummies(vgsales_df_cleaned_encoded, columns=["Genre", "ESRB_Rating"], prefix=["Genre", "ESRB"])

# Top n encode the platform
top_platforms = vgsales_df_cleaned_encoded["Platform"].value_counts().nlargest(10).index
vgsales_df_cleaned_encoded["Platform"] = vgsales_df_cleaned_encoded["Platform"].where(vgsales_df_cleaned_encoded["Platform"].isin(top_platforms), other="Other")
vgsales_df_cleaned_encoded = pd.get_dummies(vgsales_df_cleaned_encoded, columns=["Platform"], prefix="Platform")

# Frequency encode publisher and developer
for col in ["Publisher", "Developer"]:
    freq_map = vgsales_df_cleaned_encoded[col].value_counts(normalize=True)
    vgsales_df_cleaned_encoded[col + "_freq"] = vgsales_df_cleaned_encoded[col].map(freq_map)
# Drop original columns
vgsales_df_cleaned_encoded = vgsales_df_cleaned_encoded.drop(columns=["Publisher", "Developer"])

In [ ]:
vgsales_df_cleaned_encoded.sample(10)

In [ ]:
# can use this to create a df with no missing critic and user scores for better EDA later. I will keep all the other values in for now
vgsales_eda_df = vgsales_df_cleaned.dropna(subset=["Critic_Score", "User_Score"])

In [ ]:
# Showing there is a class imbalance inthe platform, since 20% are PC. but this isn't going to be a target, so it doesnt matter.
vgsales_df_cleaned["Platform"].value_counts(normalize=True)

## Conclusions:
### The data is usable and helpful. There are different success metrics such as rank, global sales, and some critics/user score. Rank is likely going to be the target variable for most models. The data was modified and corrected as needed, and a copy of the data was encoded for use later in models. There is a class imbalance for the Platform feature, where 20% of the data's games are made for PC. but we will be using this as a feature, not a target, so it is irrelevant.


# Processing the Video Game Recommendations Dataset

## Features
| Column Name     | Data Type  | Description                                                   |
|----------------|------------|---------------------------------------------------------------|
| app_id         | int64      | Unique game identifier on Steam                               |
| helpful        | int64      | How many users found the recommendation helpful               |
| funny          | int64      | How many users found the recommendation funny                 |
| date           | object     | Date of publishing                                            |
| is_recommended | bool       | Is the user recommending the product?                         |
| hours          | float64    | How many hours played by the reviewing user                   |
| user_id        | int64      | User's anonymized ID                                          |
| review_id      | int64      | Autogenerated ID for the review itself                        |
| products       | int64      | Number of games/add-ons purchased by the user                 |
| reviews        | int64      | Number of reviews published by the reviewing user             |
| title          | object     | Name of the game being reviewed                               |
| date_release   | object     | Game's date of release                                        |
| win            | bool       | Supports Windows OS                                           |
| mac            | bool       | Supports Mac OS                                               |
| linux          | bool       | Supports Linux OS                                             |
| rating         | object     | Product rating category                                       |
| positive_ratio | int64      | Ratio of positive feedbacks (%)                               |
| user_reviews   | int64      | Number of user reviews available on the Steam page            |
| price_final    | float64    | Price in US dollars ($) calculated after the discount         |
| price_original | float64    | Price in US dollars ($) before the discount                   |
| discount       | float64    | Discount percentage                                            |
| steam_deck     | bool       | Supports the Steam Deck                                       |


In [ ]:
vgrecs_merged_df.sample(10)

In [ ]:
# Acknowledge date
vgrecs_merged_df['date_release'] = pd.to_datetime(vgrecs_merged_df['date_release'], errors='coerce')

In [ ]:
# Summary Stats
vgrecs_merged_df.info()
vgrecs_merged_df.describe(include="all")

# everything looks normal, all numerical columns are within acceptable ranges

In [ ]:
# Checking columns for duplicate values
print("Duplicate rows in vgrecs:", vgrecs_merged_df.duplicated().sum())

# Check if there are any duplicated game names with the same app_id
vgrecs_merged_df[vgrecs_merged_df.duplicated(subset=["app_id", "title"], keep=False)]

# No duplicates

In [ ]:
# Check if there are any duplicated reviews
vgrecs_merged_df[vgrecs_merged_df.duplicated("review_id", keep=False)]

# None are duplicated

In [ ]:
# Mean, median, mode
df_mmm_stats(vgrecs_games_df_raw)

In [ ]:
# Video Game Recommendation Null Counts:
print(vgrecs_merged_df.isna().sum())
print(vgrecs_merged_df.shape)

# Perfect data, hallelujah
# No values to impute

In [ ]:
# Identify inconsistent/unusable data

# check for any invalid dates
print("Number of missing date_release:", vgrecs_merged_df['date_release'].isna().sum())

# verify unique values
print(vgrecs_merged_df.shape[0])
for col in ["review_id", "user_id", "app_id", "date_release", "rating"]:
    unique_vals = vgrecs_merged_df[col].unique()
    print(f"{col}: {len(unique_vals)}")

# everything looks normal here too


In [ ]:
vgrecs_merged_df

In [ ]:
# Encoding categorical variables

vgrecs_merged_df_encoded = vgrecs_merged_df.copy()

# Ordinal encode the review rating
# get unique values
vgrecs_merged_df["rating"].unique()
# create a map to encode numbers to values
rating_order = ['Overwhelmingly Negative', 'Very Negative', 'Mostly Negative', 'Negative', 'Mixed', 'Positive', 'Mostly Positive', 'Very Positive', 'Overwhelmingly Positive']
rating_map = {label: idx for idx, label in enumerate(rating_order)}
print(rating_map)
# apply the mapping (low numbers are low ratings, high numbers are high ratings - easy to rememeber)
vgrecs_merged_df_encoded['rating_encoded'] = vgrecs_merged_df_encoded['rating'].map(rating_map)


In [ ]:
# Showing there is a significant class imbalance in the rating values, since 50% are overwhelmingly positive.
# This will be taken into consideration when modelling (balancing class weights)
vgrecs_merged_df['rating'].value_counts(normalize=True)

In [ ]:
vgrecs_merged_df.columns

In [ ]:
# Showing there is also significant class imbalance in the is_recommended values, since 80% are True.
# Because anything above a "mixed" rating will likely also be recommended, this imbalance can be taken 
# into consideration when generating the model by enabling the class weights parameter or resampling the data.
vgrecs_merged_df["is_recommended"].value_counts(normalize=True)


## Conclusions

### The data is very clean and will be perfect for my use case. There are no null values, duplicates, or inconsistencies. None of the data needs to be modified in any way prior to usage, with the exception of needing to ordinally encode the review rating to a number. There is a very large class imbalance on the outputs, but we can but we can apply class weighting to ensure the model doesn't just learn the dominant class. We can also use ensemble methods like Random Forests or XGBoost, which handle imbalanced data better than naive models like regression or something.


# Processing the Olist eCommerce Dataset

## Features

| Column Name                   | Data Type | Description                                                                                      |
|------------------------------|-----------|--------------------------------------------------------------------------------------------------|
| order_id                     | object    | Unique ID of the order                                                                           |
| order_item_id                | int64     | Sequential number identifying number of items in the same order                                  |
| product_id                   | object    | Unique product identifier                                                                         |
| seller_id                    | object    | Seller unique identifier                                                                          |
| shipping_limit_date          | object    | Seller shipping limit date for handing the order to the logistic partner                         |
| price                        | float64   | Item price                                                                                        |
| freight_value                | float64   | Item freight value (split between items if multiple)                                             |
| product_category_name        | object    | Root category of product (in Portuguese)                                                         |
| product_name_lenght          | float64   | Number of characters in the product name                                                         |
| product_description_lenght  | float64   | Number of characters in the product description                                                  |
| product_photos_qty           | float64   | Number of published product photos                                                               |
| product_weight_g             | float64   | Product weight in grams                                                                          |
| product_length_cm            | float64   | Product length in centimeters                                                                    |
| product_height_cm            | float64   | Product height in centimeters                                                                    |
| product_width_cm             | float64   | Product width in centimeters                                                                     |
| product_category_name_english| object    | Product category name in English                                                                 |
| customer_id                  | object    | Key to the orders dataset; each order has a unique customer_id                                   |
| order_purchase_timestamp     | object    | Purchase timestamp                                                                                |
| review_creation_date         | object    | Date the satisfaction survey was sent to the customer                                            |
| review_answer_timestamp      | object    | Timestamp of the customer's survey response                                                      |
| customer_unique_id           | object    | Unique identifier of a customer                                                                  |
| customer_state               | object    | Customer's state/province in Brazil (2-letter abbreviation)                                      |
| review_score                 | float64   | Score from 1 to 5 given by the customer on the satisfaction survey                               |


In [ ]:
olist_cleaned_df = olist_merged_df.copy()

In [ ]:
# View the df
print(olist_cleaned_df.shape)
olist_cleaned_df.sample(10)

In [ ]:
# Summary Stats
olist_cleaned_df.info()
olist_cleaned_df.describe(include="all")

In [ ]:
# Checking for duplicate rows
print("Duplicates rows in olist:", olist_cleaned_df.duplicated().sum())

# Delete any duplicates:
olist_cleaned_df = olist_cleaned_df.drop_duplicates(keep="first")

In [ ]:
# remove rows that dont have a product_category_name
before = len(olist_cleaned_df)
olist_cleaned_df = olist_cleaned_df.dropna(subset=['product_category_name'])
after = len(olist_cleaned_df)
print(f"Removed {before - after} rows without product_category_name")


In [ ]:
# Check for nulls
print(olist_cleaned_df.isna().sum())

# looks like there are 24 missing english product category names, remove those
olist_cleaned_df = olist_cleaned_df.dropna(subset=['product_category_name_english'])

# remove the rows without a review score, that is the target
olist_cleaned_df = olist_cleaned_df.dropna(subset=['review_score'])

# One row has missing product weight/dimensions, remove that
olist_cleaned_df = olist_cleaned_df.dropna(subset=['product_weight_g'])


#check nulls again
print()
print(olist_cleaned_df.isna().sum())
# all nulls removed, yeehaw


In [ ]:
# Subtract review_creation time from order_purchase_timestamp to get time between purchase and survey sent

# both columns should be datetime
olist_cleaned_df['review_creation_date'] = pd.to_datetime(olist_cleaned_df['review_creation_date'])
olist_cleaned_df['order_purchase_timestamp'] = pd.to_datetime(olist_cleaned_df['order_purchase_timestamp'])

# Create a new column for the time difference in hours
olist_cleaned_df['days_to_review'] = (olist_cleaned_df['review_creation_date'] - olist_cleaned_df['order_purchase_timestamp'])


In [ ]:
# Checking inconsistensies

# Olist: Checking for orders delivered before purchase (which doesn"t make sense)
olist_orders_df_raw["order_purchase_timestamp"] = pd.to_datetime(olist_orders_df_raw["order_purchase_timestamp"])
olist_orders_df_raw["order_delivered_customer_date"] = pd.to_datetime(olist_orders_df_raw["order_delivered_customer_date"])
olist_orders_df_raw[
    olist_orders_df_raw["order_delivered_customer_date"] < olist_orders_df_raw["order_purchase_timestamp"]
]

# If we subtract order_purchase_timestamp from review_creation_date, it should only be a positive time delta. remove any negative ones:
num_negative = (olist_cleaned_df['days_to_review'] < pd.Timedelta(0)).sum()
print(f"Number of rows with negative days_to_review: {num_negative}")
olist_cleaned_df = olist_cleaned_df[olist_cleaned_df['days_to_review'] >= pd.Timedelta(0)]

# Product IDs shouldn"t map to multiple categories
product_counts = olist_cleaned_df.groupby("product_id")["product_category_name"].nunique()
print(product_counts.value_counts())
# no_category_id = product_counts[product_counts == 0].index

# clean up the typos in column names:
olist_cleaned_df = olist_cleaned_df.rename(columns={'product_name_lenght': 'product_name_length'})
olist_cleaned_df = olist_cleaned_df.rename(columns={'product_description_lenght': 'product_description_length'})
print(olist_cleaned_df.columns)

# Drop product_category_name since we already have the english translation
olist_cleaned_df = olist_cleaned_df.drop("product_category_name", axis=1)


In [ ]:
# Mean, Median, Mode
df_mmm_stats(olist_cleaned_df)

In [ ]:
# Encoding variables

# verify unique values
print(olist_cleaned_df.shape[0])
for col in olist_cleaned_df.columns:
    unique_vals = olist_cleaned_df[col].unique()
    print(f"{col}: {len(unique_vals)}")

olist_cleaned_df.info()


# don't need ids for modelling 
olist_cleaned_encoded_df = olist_cleaned_df.copy()
olist_cleaned_encoded_df = olist_cleaned_encoded_df.drop(["order_id", "customer_id", "product_id", "seller_id", "customer_unique_id"], axis=1)
# important times are covered by days_to_review anyway, drop them
olist_cleaned_encoded_df = olist_cleaned_encoded_df.drop(["review_creation_date", "shipping_limit_date", "review_answer_timestamp"], axis=1)

# One-hot encode categorical columns
olist_cleaned_encoded_df = pd.get_dummies(olist_cleaned_encoded_df, columns=['product_category_name_english', 'customer_state'])


# Convert timedelta to days (float)
olist_cleaned_encoded_df['days_to_review'] = olist_cleaned_encoded_df['days_to_review'].dt.total_seconds() / 86400  # seconds in a day

olist_cleaned_encoded_df.sample(10)



In [ ]:
# check for review ratings class imbalance 
print(olist_cleaned_df['review_score'].value_counts(normalize=True))

# yes, there is a class imbalance problem whereby 56% of the rows have a 5 star rating.

## Conclusions:

### The data is usable, we have 110741 rows remaining from the original 113314 rows in the merged df. None of the values are null or inconsistent data. The olist timestamps were subtracted in order to get a column of time lapsed between the order purchase and the customer satisfaction survey. Additionally, the categorical variables like product category was encoded.Given the strong class imbalance in review_score, for classification models, I would either binarize the target (good vs. bad reviews), or use class weights to prevent misclassifying smaller classes. For regression models, I can retain the full score range and ensure evaluation metrics like RMSE or MAE to get prediction quality across all scores.

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.

In [ ]:
# fake data similar to the line chart in storytelling with data page 1
years = np.array([2010, 2011, 2012, 2013, 2014, 2015])
data = {
"Arts & culture":     [80, 25, 28, 40, 32, 45],
"Education":          [20, 78, 45, 65, 63, 58],
"Health":             [65, 52, 60, 63, 68, 93],
"Human services":     [60, 85, 78, 60, 89, 55],
"Other":              [30, 30, 45, 27, 47, 30]}

# plot lines 
plt.figure(figsize=(8, 6))
for label, values in data.items():
    plt.plot(years, values, label=label)


plt.title("Non Profit Support")
plt.ylabel("Percentage of support")
plt.xlabel("Year")
# ticks for the axes
plt.xticks(years)
plt.yticks(np.arange(0, 101, 10))
plt.ylim(0, 100)
plt.legend()
plt.grid(False)
plt.tight_layout()

plt.show()
